In [457]:
# Autoreload
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from os import listdir
from tqdm import tqdm
import folium

import pickle

import random

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import accuracy_score,recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

from src import load_data as ld
from src import utils
from src import feature_exploring as fexp
from src import map_plot as mp
from src import ml_utils as mlu

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [473]:
pickled_model = pickle.load(open('results/grid_search_best_rf.pkl', 'rb'))

Lo que quiero hacer es crear un codigo que utilice el pickle para sacar la prediccion de nuevos datos. 

En estos nuevos datos que entren hay que aislar los numeros mmsi para posteriormente poder plotearlos. Eso significa crear un codigo que se asegure de entrar barcos enteros.

Eso significa que coger una sample no seria buena idea, hay que coger un grupo de datos agrupados por mmsi.

Pensar si en el codigo anterior, en vez de coger una sample habria que coger una sample de mmsi's, para asegurarse de que siempre son barcos enteros.

Comprobar la accuracy por barco tambien seria interesante! O necesario...

##### Creacion de la funcion mmmsi_split

In [438]:
df_all_gears = ld.load_data_model()

In [439]:
X = df_all_gears.drop('is_fishing',axis=1)

In [440]:
y = df_all_gears['is_fishing']

In [442]:
X_train, X_test, y_train, y_test = mlu.train_test_mmsi_split(X,y,0.8)

Train: 79.0% | Test: 21.0%


In [268]:
# falta aplicar esto al grid_search.py

##### Utilizar pickle para sacar la prediccion

In [458]:
new_data = df_all_gears.sample(10000)
### new_data = df_all_gears[df_all_gears.mmsi==df_all_gears.mmsi.unique()[92]]

# droping all the nul values (in the future, predict them)
new_data_na = new_data.dropna()

# setting a threshold to is_fishing
new_data_na['is_fishing'] = mlu.set_threshold(new_data_na,0.5)

# keep mmsi col
mmsi_col = new_data_na['mmsi']

cols = fexp.column_select(new_data_na,drop_always=True,drop_mmsi=True,drop_y=True,drop_gear=True)
new_data_cols = new_data_na[cols]

# scaling the train data and using the same object to scale the test data
new_data_cols['distance_from_port'] = utils.min_max(new_data_cols,'distance_from_port')
new_data_cols['distance_from_shore'] = utils.min_max(new_data_cols,'distance_from_shore')

In [459]:
prediction = pickled_model.predict(new_data_cols)

In [460]:
# resultado global de la prediccion

In [461]:
result = pd.DataFrame({'mmsi':mmsi_col,'is_fishing':new_data_na['is_fishing'],'prediction':prediction,'lat':new_data_na['lat'],'lon':new_data_na['lon']})

In [462]:
result

,mmsi,is_fishing,prediction,lat,lon
1553550,1.867463e+14,1,1,42.278751,15.141267
1218645,2.311543e+14,0,0,40.639359,-8.692692
1301698,2.690503e+14,0,0,64.171448,-51.721985
1847467,1.038997e+14,1,1,25.034010,166.089783
824802,8.878018e+13,0,0,49.917446,-6.317163
...,...,...,...,...,...
1262044,2.690503e+14,0,0,68.784554,16.547459
1571574,2.073022e+14,1,0,53.511486,6.266258
1832630,8.611911e+13,0,0,38.593628,-28.474886
1011559,1.710880e+14,0,0,-5.060000,-145.587860


In [463]:
y_test = list(result['is_fishing'])
y_pred_test = list(result['prediction'])

In [464]:
results = {'Accuracy': accuracy_score(y_test, y_pred_test),
           'Recall': recall_score(y_test, y_pred_test),
           'F1': f1_score(y_test, y_pred_test),
           'set': 'test'}

In [465]:
results

{'Accuracy': 0.9036,
 'Recall': 0.9130979736787131,
 'F1': 0.9006799917576757,
 'set': 'test'}

Ahora que ya tengo como generar los resultados, necesito que me saque un informe con lo siguiente:
- El resultado global de la prediccion
- El resultado de la prediccion para cada barco
- De alguna manera, un plot de los puntos de pesca y no pesca de los barcos

##### Prediccion por barco

In [467]:
# resultado de la prediccion por barco
vessel_results = []
count = 0
vessels = result['mmsi'].unique()
for i in vessels:
    vessel_results.append({'mmsi':i,
                           'number':count,
                           'F1': f1_score(result[result['mmsi']==i]['is_fishing'], result[result['mmsi']==i]['prediction'],zero_division=0)})
    count += 1

In [468]:
vessel_results

[{'mmsi': 186746307373264.0, 'number': 0, 'F1': 1.0},
 {'mmsi': 231154271004480.0, 'number': 1, 'F1': 0.9565217391304348},
 {'mmsi': 269050323939773.0, 'number': 2, 'F1': 0.9548133595284873},
 {'mmsi': 103899707980421.0, 'number': 3, 'F1': 0.8915662650602411},
 {'mmsi': 88780175381729.0, 'number': 4, 'F1': 0.5},
 {'mmsi': 222656062190286.0, 'number': 5, 'F1': 0.7999999999999999},
 {'mmsi': 274850145767759.0, 'number': 6, 'F1': 0.8229342327150084},
 {'mmsi': 240226012373593.0, 'number': 7, 'F1': 0.875},
 {'mmsi': 268433469365583.0, 'number': 8, 'F1': 0.8571428571428571},
 {'mmsi': 118485905119293.0, 'number': 9, 'F1': 0.9126760563380281},
 {'mmsi': 229926462784947.0, 'number': 10, 'F1': 0.888888888888889},
 {'mmsi': 103786638076601.0, 'number': 11, 'F1': 1.0},
 {'mmsi': 139028728648447.0, 'number': 12, 'F1': 0.0},
 {'mmsi': 91817019950540.0, 'number': 13, 'F1': 0.9876543209876543},
 {'mmsi': 141471888075398.0, 'number': 14, 'F1': 0.6666666666666666},
 {'mmsi': 63476152619655.0, 'number'

In [429]:
# mirar por que algunos barcos me dan 0! Seran los purse seines?

In [430]:
number = 101

In [469]:
vessel_number = result[result.mmsi==result.mmsi.unique()[number]]
latitudes = np.array(vessel_number['lat'])
longitudes = np.array(vessel_number['lon'])
coords = [(lat,lon) for lat,lon in zip(latitudes,longitudes)]

In [470]:
result = result[result.mmsi==result.mmsi.unique()[number]]

In [471]:
df_1 = result[result.is_fishing==1][['lat','lon']]
coords_1 = [(lat,lon) for lat,lon in zip(np.array(df_1['lat']),np.array(df_1['lon']))]

In [472]:
df_2 = result[result.prediction==1][['lat','lon']]
coords_2 = [(lat,lon) for lat,lon in zip(np.array(df_2['lat']),np.array(df_2['lon']))]

##### Plotear mapas

In [504]:
m_real = mp.folium_markers(coords_1)

In [477]:
m_real.save("results/plots/m_real.png")

In [475]:
m_predict = mp.folium_polyline(coords_2)

In [478]:
m_predict.save("results/plots/m_predict.png")

Cosas a hacer:
- Probar de hacer que el modelo coja los datos con el split por mmsi que he hecho antes
- Mirar de poner una etiqueta del gear type en los resultados para ver que tipo de barco es
- Ahora mismo mis pruebas se basan en coger otra sample (muchos datos ya los habra visto el modelo). Hay que probar con otros datos, pero como? Me reservo una parte de los datos haciendo split?

- Mirar si mi script grid_search tiene que guardarme el dataframe y los mmsi's en un pickle, para usarlos luego!

Mas cosas a hacer:
- Es importantisimo en el plot, hacer un codigo que diferencie puntos de pesca de puntos de no pesca. Ahora mismo solo estoy ploteando puntos de pesca.
- Tambien hay que programar algo que me saque todos los plots de todos los barcos, y no la mierda que tengo hecha ahora.
- Hay que hacer algo con la data que estoy cogiendo como datos nuevos para sacar resultados. Tal vez deberia hacer un split que me deje unseen data.
- Hay que hacer algo con los purse seines! Sacarlos del load data tal vez? Pensar que.